In [13]:
import numpy as np

# Load the Happy Latents (happy.npy)
happy_latents = np.load("boundaries/boundary_happy.npy")
sad_latents = np.load("boundaries/boundary_sad.npy")

# Invert the Happy Latents to create Sad Latents
angry_latent = sad_latent + (intensity * happy_latent)


# Save the Sad Latents to a new file (sad.npy)
np.save("boundaries/angry.npy", sad_latents)

print("✅ Sad latents saved as sad.npy.")


✅ Sad latents saved as sad.npy.


In [14]:
import numpy as np

# Load the Happy and Sad Latents
happy_latents = np.load("boundaries/boundary_happy.npy")
sad_latents = np.load("boundaries/boundary_sad.npy")

# Compute Angry Latent by moving from Sad toward Happy
intensity = 0.2  # Feel free to adjust this!
angry_latent = sad_latents + (intensity * (happy_latents - sad_latents))

# Save the Angry Latents to a new file
np.save("boundaries/angry.npy", angry_latent)

print("✅ Angry latents saved as angry.npy.")


✅ Angry latents saved as angry.npy.


In [20]:
import matplotlib.pyplot as plt
from PIL import Image
import torch
import numpy as np

def tensor2im(tensor):
    if tensor.dim() == 4:
        tensor = tensor[0]  # Take the first image in the batch
    tensor = tensor.cpu().detach()
    tensor = (tensor + 1.0) / 2.0  # Scale from [-1,1] to [0,1]
    tensor = tensor.clamp(0, 1)
    tensor = tensor.mul(255).byte()
    array = tensor.permute(1, 2, 0).numpy()
    return Image.fromarray(array)

# Define intensities
intensities = np.linspace(0.05, 0.3, 12)

fig, axes = plt.subplots(3, 4, figsize=(12, 9))
axes = axes.flatten()

for i, intensity in enumerate(intensities):
    # Interpolate between sad and happy latents
    angry_latent = sad_latents + intensity * (happy_latents - sad_latents)  # shape: (18, 512)
    angry_latent_tensor = torch.from_numpy(angry_latent).unsqueeze(0).float().to(DEVICE)  # shape: (1, 18, 512)

    with torch.no_grad():
        generated, _ = encoder.decoder(angry_latent_tensor, input_is_latent=True, randomize_noise=False)
        generated = encoder.face_pool(generated)
        img = tensor2im(generated)  # tensor2im handles batch dim
        img = img.resize((256, 256), Image.LANCZOS)

    axes[i].imshow(img)
    axes[i].set_title(f"Intensity {intensity:.2f}")
    axes[i].axis("off")

plt.tight_layout()
plt.savefig("emotion_interpolation_grid.png")
print("Saved as emotion_interpolation_grid.png")



Saved as emotion_interpolation_grid.png


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import torch
import numpy as np
from models.psp import pSp 
from utils.common import tensor2im  
from argparse import Namespace

# Setup device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pSp encoder
def load_encoder(e4e_ckpt, device=DEVICE):
    opts = Namespace(
        checkpoint_path=e4e_ckpt,
        device=device,
        encoder_type='Encoder4Editing',
        start_from_latent_avg=True,
        input_nc=3,
        n_styles=18,
        stylegan_size=1024,
        is_train=False,
        learn_in_w=False,
        output_size=1024,
        id_lambda=0,
        lpips_lambda=0,
        l2_lambda=1,
        w_discriminator_lambda=0,
        use_w_pool=False,
        w_pool_size=50,
        use_ballholder_loss=False,
        optim_type='adam',
        batch_size=1,
        resize_outputs=False
    )
    encoder = pSp(opts).to(device).eval()
    return encoder

# Load your pre-existing latent vectors
happy_latents = np.load('boundaries/boundary_happy.npy')
sad_latents = np.load('boundaries/boundary_sad.npy')
angry_latents = np.load('boundaries/boundary_angry.npy')
disgust_latents = np.load('boundaries/boundary_disgust.npy')
sorrow_latents = np.load('boundaries/boundary_sorrow.npy')

# Define intensities for blending
intensities = np.linspace(4, 30, 12)

# Generate new emotions by interpolating between existing ones
new_emotions = {
    "surprise": happy_latents + 0.2 * (angry_latents - happy_latents),
    "fear": sad_latents + 0.3 * (angry_latents - sad_latents),
    "joy": happy_latents + 0.15 * (disgust_latents - happy_latents),
    "disbelief": sorrow_latents + 0.25 * (sad_latents - sorrow_latents)
}

# Load encoder (ensure the correct checkpoint path)
e4e_checkpoint = "pretrained_models/e4e_ffhq_encode.pt"  
encoder = load_encoder(e4e_checkpoint, device=DEVICE)

# Generate and save emotion variations
for i, (emotion, latent) in enumerate(new_emotions.items()):
    for j, intensity in enumerate(intensities):
        # Modify latent using intensity
        emotion_latent = latent + intensity * (happy_latents - latent)  # Modify to suit your blend
        
        # Convert to torch tensor and process
        emotion_latent_tensor = torch.from_numpy(emotion_latent).unsqueeze(0).float().to(DEVICE)

        with torch.no_grad():
            generated, _ = encoder.decoder(emotion_latent_tensor, input_is_latent=True, randomize_noise=False)
            generated = encoder.face_pool(generated)
            
            # Ensure the tensor has the correct shape
            generated = generated.squeeze(0)  # Remove batch dimension
            img = tensor2im(generated).resize((256, 256), Image.LANCZOS)

        # Save each emotion variation image as a separate PNG file
        output_filename = f"{emotion}_intensity_{intensity:.2f}.png"
        img.save(output_filename)
        print(f"Saved {output_filename}")


Loading e4e over the pSp framework from checkpoint: pretrained_models/e4e_ffhq_encode.pt
Saved surprise_intensity_4.00.png
Saved surprise_intensity_6.36.png
Saved surprise_intensity_8.73.png
Saved surprise_intensity_11.09.png
Saved surprise_intensity_13.45.png
Saved surprise_intensity_15.82.png
Saved surprise_intensity_18.18.png
Saved surprise_intensity_20.55.png
Saved surprise_intensity_22.91.png
Saved surprise_intensity_25.27.png
Saved surprise_intensity_27.64.png
Saved surprise_intensity_30.00.png
Saved fear_intensity_4.00.png
Saved fear_intensity_6.36.png
Saved fear_intensity_8.73.png
Saved fear_intensity_11.09.png
Saved fear_intensity_13.45.png
Saved fear_intensity_15.82.png
Saved fear_intensity_18.18.png
Saved fear_intensity_20.55.png
Saved fear_intensity_22.91.png
Saved fear_intensity_25.27.png
Saved fear_intensity_27.64.png
Saved fear_intensity_30.00.png
Saved joy_intensity_4.00.png
Saved joy_intensity_6.36.png
Saved joy_intensity_8.73.png
Saved joy_intensity_11.09.png
Saved jo